In [1]:
import sys 
sys.path.append('..')
from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import torch as ch
from torch import Tensor
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Uniform
from torch.utils.data import DataLoader
from delphi.stats.truncated_regression import truncated_regression
from delphi.utils.datasets import TruncatedRegressionDataset 
from delphi.oracle import Left, Interval
import datetime

In [11]:
STORE_PATH = '/home/pstefanou/Uniform2'
STORE_TABLE_NAME = 'uniform'

store = Store(STORE_PATH)

store.add_table(STORE_TABLE_NAME, { 
    'known_param_mse': float,
    'unknown_param_mse': float, 
    'ols_param_mse': float,
    'alpha': float, 
    'var': float, 
})

Logging in: /home/pstefanou/Uniform2/78d57567-ab08-4a12-9ee1-8da91c30b13f


Experiment Constants

In [12]:
# regression parameters
NUM_SAMPLES, DIMS = 10000, 10
LOWER, UPPER = -1, 1
PREDICATE = Left(Tensor([1.0]))

In [ ]:
# perform each experiment a total of 10 times
for iter_ in range(1):        
    for VAR in range(1, 21):
        # generate data
        W = Uniform(LOWER, UPPER).sample(ch.Size([DIMS, 1]))
        W0 = Uniform(LOWER, UPPER).sample(ch.Size([1, 1]))
        X = Uniform(0, 1).sample(ch.Size([NUM_SAMPLES, DIMS]))
        y = X.mm(W) + W0 + Normal(ch.zeros(1), ch.sqrt(Tensor([VAR]))).sample(ch.Size([NUM_SAMPLES]))
        # truncate
        indices = PREDICATE(y).nonzero(as_tuple=False).flatten()
        y_trunc, x_trunc = y[indices], X[indices]

        # empirical linear regression
        reg = LinearRegression() 
        reg.fit(x_trunc, y_trunc)
        emp_var = ch.var(ch.from_numpy(reg.predict(x_trunc)) - y_trunc, dim=0).unsqueeze(0)

        # experiment parameters
        args = Parameters({ 
            'alpha': Tensor([y_trunc.size(0)/NUM_SAMPLES]), 
            'phi': PREDICATE, 
            'epochs': 10,
            'num_workers': 1, 
            'batch_size': 100,
            'bias': True,
            'num_samples': 100,
            'clamp': True, 
            'radius': 5.0, 
            'var_lr': 1e-1,
            'var': emp_var,
            'lr': 1e-1,
            'shuffle': True, 
            'tol': 1e-1,
        })


        # truncated linear regression with known noise variance
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=False) # dataset 
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)
        trunc_reg = truncated_regression(phi=args.phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=args.num_samples, var=args.var, device=args.device)
        results = trunc_reg.fit(S)
        w_, w0_ = results.weight.detach().cpu(), results.bias.detach().cpu()

        # truncated linear regression with unknown noise variance
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=True) # dataset 
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)
        trunc_reg = truncated_regression(phi=args.phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=args.num_samples, device=args.device)
        results = trunc_reg.fit(S)
        var_ = results.lambda_.inverse().detach().cpu()
        w, w0 = (results.v.detach()*var_).cpu(), (results.bias.detach()*var_).cpu()

        # calculate metrics 
        real_params, ols_params, unknown_params, known_params = ch.cat([W, W0]), ch.cat([data.w, data.w0.unsqueeze(0)]), ch.cat([w, w0]), ch.cat([w_.t(), w0_.unsqueeze(0)])
        unknown_param_mse, ols_param_mse, known_param_mse = mean_squared_error(unknown_params, real_params), mean_squared_error(ols_params, real_params), mean_squared_error(known_params, real_params)

        store[STORE_TABLE_NAME].append_row({ 
            'known_param_mse': known_param_mse,
            'unknown_param_mse': unknown_param_mse,
            'ols_param_mse': ols_param_mse,
            'alpha': float(args.alpha.flatten()),
            'var': float(VAR), 
        })

# close current store
store.close()

Epoch:1 | Score tensor([0.0345, 0.0189, 0.0200, 0.0191, 0.0180, 0.0221, 0.0259, 0.0288, 0.0228,
        0.0321, 0.0455]) 
Epoch:1 | Score tensor([0.0345, 0.0189, 0.0200, 0.0191, 0.0180, 0.0221, 0.0259, 0.0288, 0.0228,
        0.0321, 0.0455]) 
Epoch:1 | Score tensor([0.0345, 0.0189, 0.0200, 0.0191, 0.0180, 0.0221, 0.0259, 0.0288, 0.0228,
        0.0321, 0.0455]) 
 | Loss 0.0558 | ||:   0%|          | 0/2 [00:00<?, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f8ca4d37438>>
Traceback (most recent call last):
  File "/home/pstefanou/env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/home/pstefanou/env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.

Epoch:1 | Score tensor([0.0685, 0.0639, 0.0642, 0.0690, 0.0640, 0.0605, 0.0640, 0.0678, 0.0687,
        0.0644, 0.1326]) 
Epoch:1 | Score tensor([0.0688, 0.0628, 0.0636, 0.0694, 0.0633, 0.0605, 0.0641, 0.0681, 0.0678,
        0.0651, 0.1328]) 
Epoch:1 | Score tensor([0.0688, 0.0628, 0.0636, 0.0694, 0.0633, 0.0605, 0.0641, 0.0681, 0.0678,
        0.0651, 0.1328]) 
Epoch:1 | Score tensor([0.0687, 0.0633, 0.0637, 0.0702, 0.0630, 0.0634, 0.0666, 0.0695, 0.0681,
        0.0660, 0.1349]) 
Epoch:1 | Score tensor([0.0687, 0.0633, 0.0637, 0.0702, 0.0630, 0.0634, 0.0666, 0.0695, 0.0681,
        0.0660, 0.1349]) 
Epoch:1 | Score tensor([0.0718, 0.0643, 0.0642, 0.0707, 0.0637, 0.0635, 0.0649, 0.0688, 0.0677,
        0.0660, 0.1349]) 
Epoch:1 | Score tensor([0.0718, 0.0643, 0.0642, 0.0707, 0.0637, 0.0635, 0.0649, 0.0688, 0.0677,
        0.0660, 0.1349]) 
Epoch:1 | Score tensor([0.0714, 0.0651, 0.0646, 0.0691, 0.0613, 0.0642, 0.0646, 0.0698, 0.0682,
        0.0677, 0.1347]) 
Epoch:1 | Score tensor([

Epoch:1 | Score tensor([ 0.0754,  0.0889,  0.0675,  0.0851,  0.0941,  0.0542,  0.0837,  0.0823,
         0.0569,  0.0821,  0.1570, -0.2476]) 
Epoch:1 | Score tensor([ 0.0754,  0.0889,  0.0675,  0.0851,  0.0941,  0.0542,  0.0837,  0.0823,
         0.0569,  0.0821,  0.1570, -0.2476]) 
Epoch:1 | Score tensor([ 0.0754,  0.0948,  0.0747,  0.0818,  0.0971,  0.0592,  0.0891,  0.0890,
         0.0626,  0.0826,  0.1631, -0.2531]) 
Epoch:1 | Score tensor([ 0.0754,  0.0948,  0.0747,  0.0818,  0.0971,  0.0592,  0.0891,  0.0890,
         0.0626,  0.0826,  0.1631, -0.2531]) 
Epoch:1 | Score tensor([ 0.0847,  0.1036,  0.0787,  0.0867,  0.1011,  0.0658,  0.0852,  0.0902,
         0.0737,  0.0820,  0.1702, -0.2732]) 
Epoch:1 | Score tensor([ 0.0847,  0.1036,  0.0787,  0.0867,  0.1011,  0.0658,  0.0852,  0.0902,
         0.0737,  0.0820,  0.1702, -0.2732]) 
Epoch:1 | Score tensor([ 0.0869,  0.1089,  0.0895,  0.0909,  0.1100,  0.0779,  0.0909,  0.0982,
         0.0872,  0.0879,  0.1892, -0.3197]) 
Epoch:

Epoch:1 | Score tensor([ 0.0847,  0.0872,  0.0859,  0.0888,  0.1029,  0.0854,  0.0839,  0.0861,
         0.0843,  0.0821,  0.1709, -0.2759]) 
Epoch:1 | Score tensor([ 0.0847,  0.0872,  0.0859,  0.0888,  0.1029,  0.0854,  0.0839,  0.0861,
         0.0843,  0.0821,  0.1709, -0.2759]) 
Epoch:1 | Score tensor([ 0.0823,  0.0854,  0.0832,  0.0858,  0.0999,  0.0828,  0.0826,  0.0842,
         0.0840,  0.0793,  0.1671, -0.2650]) 
Epoch:1 | Score tensor([ 0.0823,  0.0854,  0.0832,  0.0858,  0.0999,  0.0828,  0.0826,  0.0842,
         0.0840,  0.0793,  0.1671, -0.2650]) 
Epoch:1 | Score tensor([ 0.0809,  0.0838,  0.0814,  0.0818,  0.0972,  0.0800,  0.0797,  0.0808,
         0.0829,  0.0772,  0.1620, -0.2533]) 
Epoch:1 | Score tensor([ 0.0809,  0.0838,  0.0814,  0.0818,  0.0972,  0.0800,  0.0797,  0.0808,
         0.0829,  0.0772,  0.1620, -0.2533]) 
Epoch:1 | Score tensor([ 0.0792,  0.0821,  0.0794,  0.0781,  0.0932,  0.0771,  0.0787,  0.0782,
         0.0811,  0.0733,  0.1570, -0.2410]) 
Epoch:

Epoch:1 | Score tensor([0.0835, 0.0824, 0.0740, 0.0884, 0.0794, 0.0878, 0.0766, 0.0793, 0.0773,
        0.0866, 0.1626]) 
Epoch:1 | Score tensor([0.0835, 0.0824, 0.0740, 0.0884, 0.0794, 0.0878, 0.0766, 0.0793, 0.0773,
        0.0866, 0.1626]) 
Epoch:1 | Score tensor([0.0889, 0.0787, 0.0793, 0.0922, 0.0778, 0.0869, 0.0851, 0.0877, 0.0815,
        0.0821, 0.1687]) 
Epoch:1 | Score tensor([0.0889, 0.0787, 0.0793, 0.0922, 0.0778, 0.0869, 0.0851, 0.0877, 0.0815,
        0.0821, 0.1687]) 
Epoch:1 | Score tensor([0.0881, 0.0836, 0.0798, 0.0900, 0.0809, 0.0886, 0.0866, 0.0859, 0.0844,
        0.0820, 0.1707]) 
Epoch:1 | Score tensor([0.0881, 0.0836, 0.0798, 0.0900, 0.0809, 0.0886, 0.0866, 0.0859, 0.0844,
        0.0820, 0.1707]) 
Epoch:1 | Score tensor([0.0881, 0.0836, 0.0798, 0.0900, 0.0809, 0.0886, 0.0866, 0.0859, 0.0844,
        0.0820, 0.1707]) 
Epoch:1 | Score tensor([0.0889, 0.0860, 0.0841, 0.0895, 0.0850, 0.0915, 0.0872, 0.0914, 0.0882,
        0.0851, 0.1773]) 
Epoch:1 | Score tensor([

Epoch:1 | Score tensor([0.0881, 0.0906, 0.0859, 0.0907, 0.0883, 0.0939, 0.0893, 0.0876, 0.0897,
        0.0869, 0.1788]) 
Epoch:1 | Score tensor([0.0881, 0.0906, 0.0859, 0.0907, 0.0883, 0.0939, 0.0893, 0.0876, 0.0897,
        0.0869, 0.1788]) 
Epoch:1 | Score tensor([0.0896, 0.0920, 0.0879, 0.0911, 0.0906, 0.0962, 0.0910, 0.0902, 0.0917,
        0.0887, 0.1824]) 
Epoch:1 | Score tensor([0.0896, 0.0920, 0.0879, 0.0911, 0.0906, 0.0962, 0.0910, 0.0902, 0.0917,
        0.0887, 0.1824]) 
Epoch:1 | Score tensor([0.0919, 0.0948, 0.0907, 0.0942, 0.0928, 0.0982, 0.0944, 0.0938, 0.0946,
        0.0931, 0.1880]) 
Epoch:1 | Score tensor([0.0919, 0.0948, 0.0907, 0.0942, 0.0928, 0.0982, 0.0944, 0.0938, 0.0946,
        0.0931, 0.1880]) 
Epoch:1 | Score tensor([0.0929, 0.0959, 0.0923, 0.0947, 0.0943, 0.0984, 0.0950, 0.0951, 0.0959,
        0.0943, 0.1900]) 
Epoch:1 | Score tensor([0.0929, 0.0959, 0.0923, 0.0947, 0.0943, 0.0984, 0.0950, 0.0951, 0.0959,
        0.0943, 0.1900]) 
Epoch:1 | Score tensor([

Epoch:1 | Score tensor([ 0.0921,  0.1073,  0.1106,  0.0980,  0.1098,  0.1230,  0.1188,  0.1030,
         0.1062,  0.1066,  0.2118, -0.3817]) 
Epoch:1 | Score tensor([ 0.0862,  0.1078,  0.1055,  0.1006,  0.1081,  0.1199,  0.1110,  0.1014,
         0.1004,  0.1025,  0.2058, -0.3661]) 
Epoch:1 | Score tensor([ 0.0862,  0.1078,  0.1055,  0.1006,  0.1081,  0.1199,  0.1110,  0.1014,
         0.1004,  0.1025,  0.2058, -0.3661]) 
Epoch:1 | Score tensor([ 0.0868,  0.1047,  0.1070,  0.0948,  0.1069,  0.1195,  0.1114,  0.0966,
         0.1008,  0.1034,  0.2021, -0.3529]) 
Epoch:1 | Score tensor([ 0.0868,  0.1047,  0.1070,  0.0948,  0.1069,  0.1195,  0.1114,  0.0966,
         0.1008,  0.1034,  0.2021, -0.3529]) 
Epoch:1 | Score tensor([ 0.0894,  0.1052,  0.1067,  0.0961,  0.1081,  0.1189,  0.1144,  0.0986,
         0.1013,  0.1025,  0.2056, -0.3590]) 
Epoch:1 | Score tensor([ 0.0894,  0.1052,  0.1067,  0.0961,  0.1081,  0.1189,  0.1144,  0.0986,
         0.1013,  0.1025,  0.2056, -0.3590]) 
Epoch:

Epoch:1 | Score tensor([ 0.0954,  0.1048,  0.0999,  0.0988,  0.1088,  0.1080,  0.0993,  0.0962,
         0.1020,  0.0972,  0.2004, -0.3545]) 
Epoch:1 | Score tensor([ 0.0962,  0.1042,  0.1010,  0.0994,  0.1076,  0.1075,  0.1003,  0.0968,
         0.1025,  0.0979,  0.2002, -0.3538]) 
Epoch:1 | Score tensor([ 0.0962,  0.1042,  0.1010,  0.0994,  0.1076,  0.1075,  0.1003,  0.0968,
         0.1025,  0.0979,  0.2002, -0.3538]) 
Epoch:1 | Score tensor([ 0.0965,  0.1046,  0.1009,  0.1003,  0.1072,  0.1062,  0.1007,  0.0968,
         0.1027,  0.0972,  0.2002, -0.3538]) 
Epoch:1 | Score tensor([ 0.0965,  0.1046,  0.1009,  0.1003,  0.1072,  0.1062,  0.1007,  0.0968,
         0.1027,  0.0972,  0.2002, -0.3538]) 
Epoch:1 | Score tensor([ 0.0965,  0.1046,  0.1009,  0.1003,  0.1072,  0.1062,  0.1007,  0.0968,
         0.1027,  0.0972,  0.2002, -0.3538]) 
 | Loss 0.5478 | ||: 100%|██████████| 48/48 [00:00<00:00, 52.19it/s]
  0%|          | 0/48 [00:00<?, ?it/s]Exception ignored in: <bound method _Mult

Epoch:1 | Score tensor([0.1123, 0.1109, 0.1204, 0.1064, 0.1013, 0.1131, 0.1036, 0.1015, 0.1219,
        0.1259, 0.2205]) 
Epoch:1 | Score tensor([0.1123, 0.1109, 0.1204, 0.1064, 0.1013, 0.1131, 0.1036, 0.1015, 0.1219,
        0.1259, 0.2205]) 
Epoch:1 | Score tensor([0.1172, 0.1139, 0.1266, 0.1123, 0.1065, 0.1177, 0.1120, 0.1071, 0.1224,
        0.1285, 0.2294]) 
Epoch:1 | Score tensor([0.1172, 0.1139, 0.1266, 0.1123, 0.1065, 0.1177, 0.1120, 0.1071, 0.1224,
        0.1285, 0.2294]) 
Epoch:1 | Score tensor([0.1115, 0.1125, 0.1230, 0.1047, 0.1023, 0.1140, 0.1104, 0.1041, 0.1164,
        0.1282, 0.2217]) 
Epoch:1 | Score tensor([0.1115, 0.1125, 0.1230, 0.1047, 0.1023, 0.1140, 0.1104, 0.1041, 0.1164,
        0.1282, 0.2217]) 
Epoch:1 | Score tensor([0.1073, 0.1105, 0.1200, 0.1023, 0.0987, 0.1075, 0.1070, 0.1010, 0.1125,
        0.1203, 0.2124]) 
Epoch:1 | Score tensor([0.1073, 0.1105, 0.1200, 0.1023, 0.0987, 0.1075, 0.1070, 0.1010, 0.1125,
        0.1203, 0.2124]) 
Epoch:1 | Score tensor([

Epoch:2 | Score tensor([0.1627, 0.1958, 0.1665, 0.1327, 0.1259, 0.1265, 0.1424, 0.1239, 0.1278,
        0.1374, 0.2840]) 
Epoch:2 | Score tensor([0.1280, 0.1728, 0.1207, 0.1071, 0.1362, 0.1421, 0.1656, 0.1303, 0.1630,
        0.1379, 0.2736]) 
Epoch:2 | Score tensor([0.1280, 0.1728, 0.1207, 0.1071, 0.1362, 0.1421, 0.1656, 0.1303, 0.1630,
        0.1379, 0.2736]) 
Epoch:2 | Score tensor([0.1140, 0.1092, 0.0980, 0.0703, 0.1076, 0.1050, 0.1248, 0.0918, 0.1328,
        0.1302, 0.2172]) 
Epoch:2 | Score tensor([0.1140, 0.1092, 0.0980, 0.0703, 0.1076, 0.1050, 0.1248, 0.0918, 0.1328,
        0.1302, 0.2172]) 
Epoch:2 | Score tensor([0.0989, 0.1101, 0.0812, 0.0708, 0.0939, 0.0856, 0.1022, 0.0837, 0.1136,
        0.1161, 0.2060]) 
Epoch:2 | Score tensor([0.0989, 0.1101, 0.0812, 0.0708, 0.0939, 0.0856, 0.1022, 0.0837, 0.1136,
        0.1161, 0.2060]) 
Epoch:2 | Score tensor([0.1086, 0.0864, 0.0839, 0.0702, 0.0757, 0.0977, 0.0875, 0.0871, 0.1196,
        0.1015, 0.1982]) 
Epoch:2 | Score tensor([

In [5]:
print("unknown mse: {}".format(unknown_param_mse))

unknown mse: 6.804898262023926


In [6]:
unknown_params

tensor([[-2.0933],
        [-1.7039],
        [-2.0965],
        [-1.9699],
        [-3.3390],
        [-1.8514],
        [-2.2874],
        [-1.5977],
        [-3.2056],
        [-2.2125],
        [-4.5427]])

In [7]:
ols_param_mse

1.5532355

In [10]:
store.close()
reader = CollectionReader(STORE_PATH)
results = reader.df(STORE_TABLE_NAME)
results.head()

  0%|          | 0/1 [00:00<?, ?it/s]


OSError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1400, in H5F__open
    unable to open file
  File "H5Fint.c", line 1615, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 941, in H5FD_sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file '/home/pstefanou/Uniform/6c5a334f-b3f1-4d57-81f2-4d59dd51ef80/store.h5'

In [9]:
len(results)

TypeError: object of type 'LinearUnknownVariance' has no len()

In [ ]:
known, unknown, ols = results[['known_param_mse', 'var']], results[['unknown_param_mse', 'var']], results[['ols_param_mse', 'var']]

In [ ]:
# plot results for regression parameter MSE
sns.lineplot(data=unknown, x='var', y='unknown_param_mse', label='unknown', color='blue')
sns.lineplot(data=known, x='var', y='known_param_mse', label='known', color='purple')
ax = sns.lineplot(data=ols, x='var', y='ols_param_mse', label='ols', color='red')
ax.set(xlabel='Noise Variance', ylabel='Squared Error')
plt.show()
# close reader
reader.close()

In [21]:
store.close()

In [ ]:
print("hi")

In [10]:
store.close()

torch.Size([10, 10000])